In [1]:
import time
import urllib.request
from urllib.request import urlopen
from optparse import OptionParser
import requests, zipfile, io
import gzip
import io

In [2]:
start = time.time()
start

1591651936.3395028

In [3]:
little = -1
#urlGo = 'http://geneontology.org/gene-associations/'
urlGo = 'http://current.geneontology.org/annotations/goa_human.gaf.gz'
urlHpo = 'http://compbio.charite.de/jenkins/job/hpo.annotations/lastBuild/artifact/util/annotation/genes_to_phenotype.txt';
#urlHpo = 'http://compbio.charite.de/jenkins/job/hpo.annotations.monthly/lastSuccessfulBuild/artifact/annotation/ALL_SOURCES_ALL_FREQUENCIES_genes_to_phenotype.txt'

#try:
#    opts, args = getopt.getopt(sys.argv[1:],"hi:o:",["ifile=","ofile="])
    
#    for opt, arg in opts:
#        if opt == '-h':
#            print ('usage')
#            sys.exit()
#        elif opt in ("-i", "--ifile"):
#            urlGo = arg
#        elif opt in ("-o", "--ofile"):
#            urlHpo = arg
#        elif opt in ("-s", "--short")
#            little = arg

#except getopt.GetoptError:
#    print ('some mistake')

In [4]:
print('Preparing ontology files. . . ')

print('Beginning file GO download with urllib2...')
filenameGO = "goa_human.gaf.gz"
path = 'C:/Users/Aco/Desktop/annovar - jupyter'
fileGOpath = "goa_human.gaf"
#urllib.request.urlretrieve(urlGo, path + '/' + filenameGO)

print ('GO file has been downloaded')


Preparing ontology files. . . 
Beginning file GO download with urllib2...
GO file has been downloaded


In [5]:

print('Beginning file HPO download with urllib2...')
filenameHpo = 'allHPODownload.txt'
#urllib.request.urlretrieve(urlHpo, path + '/' + filenameHpo)
print ('HPO file has been downloaded')

Beginning file HPO download with urllib2...
HPO file has been downloaded


In [6]:
output = open("output.txt", "r+")
outputNrna = open("outputNrnaExonic.txt", "r+")

In [7]:
#parse goa_human.gaf file with GO info
def getGOdata(filename):
    geneGO = {}
    count = 0
    with gzip.open(filename, 'rb') as goaHuman:
        for line in goaHuman:
            line = line.decode()
            pos = line.find("GO:")
            firstTab = line.find("\t")
            secondTab = line.find("\t", firstTab+1)
            thirdTab = line.find("\t", secondTab+1)
            gene = line[secondTab+1:thirdTab]
            val = thirdTab+1
            function = "go tooltip"
            if pos != -1:
                go = line[pos:pos+10]
                #if line[val:pos].strip() == "NOT":
                    #go = "NOT " + go #rather not 'not go functions' go function is better
                
                #functionWhole = line[thirdTab:endTab] + '<br />'

                if gene in geneGO:
                    current = [currentGeneInfo[0] for currentGeneInfo in geneGO[gene]]
                    if go not in current:
                        geneGO[gene].append((go, function))
                    #geneGO[gene].append((go, functionWhole))
                else:
                    geneGO[gene] = [(go, function)]
                    #print(str(pos) + " :prvi" + go)

            #if count > 30:
               #break

    print ("goa_human.gaf data: " + str(len(geneGO.keys())))
    return geneGO

#getGOdata("goa_human.gaf.gz")

In [8]:
#parse file with HPO info
def getHPOdata(filename):
    #phenotypes
    geneHpo = {}

    with open(filename, "r") as allHpo:
        for line in allHpo:
            pos = line.find("\t")
            if pos == -1:
                continue
            endOfGene = line.find("\t", pos+1)
            gene = line[pos+1:endOfGene]

            hpoStart = endOfGene+1
            hpoEnd = hpoStart+10
            hpo = line[hpoStart : hpoEnd]

            #phenotypeStart = hpoEnd+1
            #phenotypeEnd = line.find("\t", phenotypeStart+1)
            #phenotype = line[phenotypeStart : phenotypeEnd]
            phenotype = "phenotype tooltip"
            
            if gene in geneHpo:
                currentHpo = [currentGeneInfoH[0] for currentGeneInfoH in geneHpo[gene]]
                if hpo not in currentHpo:
                    geneHpo[gene].append((hpo, phenotype))
            else:
                geneHpo[gene] = [(hpo, phenotype)]
            #print(gene+" " + hpo  + " " + phenotype)
    return geneHpo


In [9]:
#parse info from annotation file
def getVariationsData(fileName):
    
    variations = {}
    allExonicGenes = ''
    geneInfo = {}

    annovarGenes = open("annovarGenes.txt", "w+")

    with open(fileName, 'r') as varFile:
        for line in varFile:
            pos = line.find("\texonic")
            if pos != -1:
                pos = pos + 1
                currentGenePositionEnd = line.find("\t", pos+7) #pos+7 is start
                currentGene = line[pos+7:currentGenePositionEnd]
                
                begin = -1
                #for i in range(0, 9): 
                #    val = line.find("\t", val + 1)
                #line[:val].replace("\t", "</td><td>")
                begin = line.find("\t")
                chro = line[:begin]
                end = line.find("\t", begin+1)
                startVar = line[begin+1:end]
                begin = end+1
                end = line.find("\t", begin)
                endVar = line[begin:end]
                begin = end+1
                end = line.find("\t", begin)
                ref = line[begin:end]
                begin = end+1
                end = line.find("\t", begin)
                alt = line[begin:end]
                #exonic, currentGene
                geneDetailEnd = line.find("\t", currentGenePositionEnd + 1)
                #geneDetail = line[currentGenePositionEnd+1 : geneDetailEnd]
                
                exonicFuncEnd = line.find("\t", geneDetailEnd + 1)
                exonicFunc = line[geneDetailEnd + 1 : exonicFuncEnd]
                
                if currentGene in variations:
                    variations[currentGene].append((startVar, endVar, ref, alt, exonicFunc, chro))#, geneDetail, exonicFunc))
                else:
                    variations[currentGene] = [(startVar, endVar, ref, alt, exonicFunc, chro)] #geneDetail, exonicFunc)]
    
    annovarGenes.write(" ".join(str(g) for g in variations.keys()))
    print(str(len(variations.keys())))
    return variations

In [10]:
#variations = getVariationsData("momAll.txt")

In [11]:
def createOutput(fileName, goFile = "goa_human.gaf.gz", hpoFile = "allHPODownload.txt"):
    
    #goFile = "goa_human.gaf.gz"
    #hpoFile = "allHPODownload.txt"
    
    geneGO = getGOdata(goFile)
    geneHpo = getHPOdata(hpoFile)
    variations = getVariationsData(fileName)
    print ("Variations: " + str(len(variations.keys())))
    
    outputFileName = "outputHtml" + fileName[:fileName.find('.')] + "200test5.1.html"
    outputHtml = open(outputFileName, "w+")
    
    outputHtml.write("<!DOCTYPE html>\n")
    outputHtml.write("<html><body><head>")
    outputHtml.write("<script src=\"https://ajax.googleapis.com/ajax/libs/jquery/3.4.1/jquery.min.js\"></script>")
    
    #datatable
    outputHtml.write("<script type=\"text/javascript\" charset=\"utf8\" src=\"https://cdn.datatables.net/1.10.20/js/jquery.dataTables.js\"></script>\n")
    outputHtml.write("<link rel=\"stylesheet\" type=\"text/css\" href=\"https://cdn.datatables.net/1.10.20/css/jquery.dataTables.css\">\n")
    
    #checkbox
    #outputHtml.write("<script type=\"text/javascript\" charset=\"utf8\" src=\"https://cdn.datatables.net/select/1.3.1/js/dataTables.select.min.js\"></script>")
    #outputHtml.write("<link rel=\"stylesheet\" type=\"text/css\" href=\"https://cdn.datatables.net/select/1.3.1/css/select.dataTables.min.css\">")
    outputHtml.write("<script type=\"text/javascript\" charset=\"utf8\" src=\"https://gyrocode.github.io/jquery-datatables-checkboxes/1.2.11/js/dataTables.checkboxes.min.js\"></script>\n")
    outputHtml.write("<link rel=\"stylesheet\" type=\"text/css\" href=\"https://gyrocode.github.io/jquery-datatables-checkboxes/1.2.11/css/dataTables.checkboxes.css\">\n")
    
    #include bootstrap for tooltip
    #outputHtml.write("<script src=\"https://maxcdn.bootstrapcdn.com/bootstrap/3.4.1/js/bootstrap.min.js\"></script>")
    #outputHtml.write("<link rel=\"stylesheet\" href=\"https://maxcdn.bootstrapcdn.com/bootstrap/3.4.1/css/bootstrap.min.css\">")
    
    #include tippy for tooltip
    outputHtml.write("<script src=\"https://unpkg.com/@popperjs/core@2/dist/umd/popper.min.js\"></script>")
    outputHtml.write("<script src=\"https://unpkg.com/tippy.js@6/dist/tippy-bundle.umd.js\"></script>")

    outputHtml.write("<link rel=\"stylesheet\" href=\"cssFile.css\">")
    outputHtml.write("<script src='scriptFile.js'></script></head>")
    outputHtml.write("")
    outputHtml.write("<div style=\"text-align:center\"><span style=\"font-size:26px;\">Function and phenotype terms associated with human exonic variations</span><br />")
    outputHtml.write("<span style=\"font-size:18px\">GO version: 2020-03-23. <br /> HPO version: #1271 Mar 27, 2020</span></div>")
    #export button
    outputHtml.write("<div id=\"firstXrows\"><label for=\"numberOfRows\">Select first</label> <input type=\"number\" id=\"rowsNumber\" name=\"numberOfRows\" style=\"width:50px\" min = 1> rows</div>")
    outputHtml.write("<div><input type=\"button\" value=\"Export selected rows to CSV\" id=\"exportcsv\" onclick=\"exportCSV()\"></div>")
    
    outputHtml.write("<div id=\"showXYfunctions\">Show  <select id=\"showXFunctions\"><option value=\"10\">10</option><option value=\"20\">20</option><option value=\"30\">30</option><option value=\"all\">all</option></select>functions</div>")
    outputHtml.write("<div class=\"dataTables_length\"></div>")
    outputHtml.write("<table id=\"annotationTable\" class=\"display cell-border\" style=\"text-align:center\">")
    outputHtml.write("<thead><tr><th class=\" select-checkbox\"</th>")
    outputHtml.write("<th>Chr</th><th>Start</th><th>End</th><th>Ref</th><th>Alt</th><th>ExonicFunc.refGene</th><th>Gene.refGene</th><th>GO associations</th><th>HPO functions</th><th>All GO</th><th>All HPO</th></tr></thead><tbody>")
    
    startURL = "<a href = \"https://www.ebi.ac.uk/QuickGO/term/"
    startURLhpo = "<a href = \"https://hpo.jax.org/app/browse/term/"
    aTagOpen = "<a"
    aTagClosed = "</a>"

    counterExonic = 0
    counterNrnaExonic = 0
    counterNo = 0
    counter = 0
    allExonicGenes = ""
    #list of all gene IDs for Uniprot convert
    
    noGo = open("genesWithoutGO.txt", "w+")
    
    with open(fileName, 'r') as file:
        for line in file:
            #line = file.readline()
            pos = line.find("\texonic")
            if pos != -1:
                pos = pos + 1
                currentGenePositionEnd = line.find("\t", pos+7) #pos+7 is start
                currentGene = line[pos+7:currentGenePositionEnd]
                if allExonicGenes.find(currentGene) == -1:
                    #goNo, hpoNo, ncRna = False
                    if counter > 200:
                        break
                    counter = counter + 1
                    allExonicGenes = allExonicGenes + currentGene
                    
                    for j in range (0, len(variations[currentGene])):
                        outputHtml.write("<tr><td></td><td>") #first column for checkbox
                        val = -1                                   

                        #column chr
                        outputHtml.write(variations[currentGene][j][5] + "</td><td>")
                        
                        #columns start, end, ref, alt
                        for i in range(0, 5):
                        #outputHtml.write(" ".join(str(e[i] + "<br />") for e in variations[currentGene]))
                            outputHtml.write(variations[currentGene][j][i] + "</td><td>")
                        
                        #outputHtml.write("</td><td>")

                        outputHtml.write("<span data-tippy-content=\"" + currentGene + "\">")

                        #column gene
                        outputHtml.write(currentGene + "</span></td><td>")

                    ##geneDetail
                    #outputHtml.write(variations[currentGene][0][5] + "</td><td>")

                    #exonic.Func                
                    #outputHtml.write(" ".join(str(e[5] + "<br />") for e in variations[currentGene]))
                    #outputHtml.write("</td><td>")

                    #if line[pos-1:pos] == "\t":
                        #counterExonic = counterExonic + 1
                        #output.write(currentGene)
                        #output.write(" ")
                    #else:
                        #counterNrnaExonic = counterNrnaExonic + 1
                        #outputNrna.write(currentGene)
                        #outputNrna.write(" ")
                        #ncRna = True                    

                        if currentGene in geneGO:
                            #outputHtml.write("*".join(str(startURL + e[0] +"\" target=\"_blank\">" + e[1] + aTagClosed)  for e in geneGO[currentGene])+ " ")#[:5]) + " ") #GO
                            if len(geneGO[currentGene]) > 10:
                                #bootstrap tooltip
                                #outputHtml.write(" ".join(str(startURL + e[0] + "\" data-toggle=\"tooltip\" title=\"" + e[1] + "\"" + "\" target=\"_blank\">" + e[0] + "<br />" + aTagClosed)  for e in geneGO[currentGene][:5]) + " ") #GO
                                #outputHtml.write(" ".join(str(startURL + e[0] + "\" data-toggle=\"tooltip\" title=\"" + e[1] + "\"" + "\" target=\"_blank\" style=\"display:none\">" + e[0] + "<br /> " +aTagClosed)  for e in geneGO[currentGene][5:]) + " ") #GO

                                #tippy tooltip
                                outputHtml.write(" ".join(str(startURL + e[0] + "\" data-tippy-content=\""+ e[1] + "\" target=\"_blank\">" + e[0] + "<br />" + aTagClosed)  for e in geneGO[currentGene][:10]) + " ") #GO
                                outputHtml.write(" ".join(str(startURL + e[0] + "\" data-tippy-content=\""+ e[1] + "\" target=\"_blank\" style=\"display:none\">" + e[0] + "<br /> " +aTagClosed)  for e in geneGO[currentGene][10:]) + " ") #GO
                                #outputHtml.write("<datalist id="+"GOs>" + " ".join(str("<option value=" + "<a href = https://www.ebi.ac.uk/QuickGO/term/" + e + ">" + e + "</a> >") for e in geneGO[currentGene][5:]) + "</datalist>") 
                            elif len(geneGO[currentGene]) >= 1:
                                outputHtml.write(" ".join(str(startURL + e[0] + "\" data-tippy-content=\""+ e[1] + "\" target=\"_blank\">" + e[0] + "<br /> " + aTagClosed)  for e in geneGO[currentGene][:10]) + " ") #GO
                        else:
                            noGo.write(" " + currentGene)

                        outputHtml.write("</td><td>")

                        if currentGene in geneHpo:
                            if len(geneHpo[currentGene]) > 10:
                                outputHtml.write(" ".join(str(startURLhpo + e[0] + "\" data-tippy-content=\""+ e[1] + "\" target=\"_blank\">" + e[0] + "<br />" + aTagClosed)  for e in geneHpo[currentGene][:10]) + " ")
                                outputHtml.write(" ".join(str(startURLhpo + e[0] + "\" data-tippy-content=\""+ e[1] + "\" target=\"_blank\" style=\"display:none\">" + e[0] + "<br /> " +aTagClosed)  for e in geneHpo[currentGene][10:]) + " ") #GO
                                #outputHtml.write(" ".join(str(startURLhpo + e +"\" target=\"_blank\" style=\"display:none\">" + e + "<br /> " +aTagClosed) for e in geneHpo[currentGene][5:]))
                            elif len(geneHpo[currentGene]) >=1:
                                outputHtml.write("<br />".join(str(startURLhpo + e[0] + "\" data-tippy-content=" + e[1] + "\" target=\"_blank\">" + e[0] + aTagClosed)  for e in geneHpo[currentGene][:10]) + " ") #GO
                                #outputHtml.write(" ".join(str(startURLhpo + e +"\" target=\"_blank\">" + e + aTagClosed) for e in geneHpo[currentGene][:5]))
                                #outputHtml.write("</td><td>")
                            #else:
                                #outputHtml.write("no info")
                                #hpoNo = True
                                #if goNo and ncRna:
                                    #counterNo = counterNo + 1

                        outputHtml.write("</td><td><a href=\"https://www.ebi.ac.uk/QuickGO/annotations?geneProductId=O14764\" target=\"_blank\"><img src=\"icon1.png\" style=\"width:50%; height:50%\"></a></td><td><a href=\"https://hpo.jax.org/app/browse/gene/2563\" target=\"_blank\"><img src=\"icon2.png\"></a></td></tr>\n")

    endOfTable = "<tfoot><tr><th></th><th>Chr</th><th>Start</th><th>End</th><th>Ref</th><th>Alt</th><th>ExonicFunc.refGene</th><th>Gene.refGene</th><th>GO associations</th><th>HPO functions</th><th>All GO</th><th>All HPO</th></tr></tfoot></table>"
    outputHtml.write("</tbody>")
    outputHtml.write(endOfTable)
    outputHtml.write("</body></html>")


In [12]:
import os, string

print("Calling annovar")

#my_cmd = "perl table_annovar.pl example/ex1.avinput humandb/ -buildver hg19 -out myanno -remove -protocol refGene,cytoBand,dbnsfp30a -operation g,r,f -nastring . -csvout -polish -xreffile example/gene_fullxref.txt"
#my_cmd = "perl table_annovar.pl mom/mom.vcf humandb/ -buildver hg19 -out mom -remove -protocol refGene -operation g -nastring . -vcfinput -polish
#my_cmd_output = os.popen(my_cmd)

print ("Annovar finished")

Calling annovar
Annovar finished


In [13]:
#createOutput("momAll.txt")
#createOutput("dadAll.txt")
#createOutput("sonAll.txt")
createOutput("mom.hg19_multianno.txt")

goa_human.gaf data: 19691
8268
Variations: 8268


In [14]:
duration = round(time.time() - start, 2)
print("Duration: " + str(duration) + "s")

Duration: 32.0s
